In [ ]:
########################################################################################################################
# Filename: threshold_learning_demo.ipynb
#
# Purpose: Demonstrate the use of the 'threshold_learning' library for use in multi-label
#          classification tasks.
#
# Author(s): Bobby (Robert) Lumpkin
#
# Library Dependencies: numpy, pandas, tensorflow, bpmll, threshold_learning
########################################################################################################################

# Threshold Learning Demonstration for Multi-Label Classification

In [21]:
## Import necessary modules
from scipy.io import arff
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import bpmll

In [3]:
## Load the 'Yeast' dataset
data_train = arff.loadarff('Yeast/Yeast-train.arff')
df_train = pd.DataFrame(data_train[0])

df_train.head()

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,...,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,Class14
0,0.093700,0.139771,0.062774,0.007698,0.083873,-0.119156,0.073305,0.005510,0.027523,0.043477,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
1,-0.022711,-0.050504,-0.035691,-0.065434,-0.084316,-0.378560,0.038212,0.085770,0.182613,-0.055544,...,b'0',b'0',b'1',b'1',b'0',b'0',b'0',b'1',b'1',b'0'
2,-0.090407,0.021198,0.208712,0.102752,0.119315,0.041729,-0.021728,0.019603,-0.063853,-0.053756,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'1',b'1',b'0'
3,-0.085235,0.009540,-0.013228,0.094063,-0.013592,-0.030719,-0.116062,-0.131674,-0.165448,-0.123053,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'1',b'1',b'1'
4,-0.088765,-0.026743,0.002075,-0.043819,-0.005465,0.004306,-0.055865,-0.071484,-0.159025,-0.111348,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'


In [4]:
df_train.columns

Index(['Att1', 'Att2', 'Att3', 'Att4', 'Att5', 'Att6', 'Att7', 'Att8', 'Att9',
       'Att10',
       ...
       'Class5', 'Class6', 'Class7', 'Class8', 'Class9', 'Class10', 'Class11',
       'Class12', 'Class13', 'Class14'],
      dtype='object', length=117)

In [ ]:
###### Convert the class labels into integers
label_names = []
for name in df_train.columns:
    if "Class" in name:
        label_names.append(name)
df_train[label_names] = df_train[label_names].astype("int32")
df_train.head()

In [31]:
X_train = df_train[np.setdiff1d(df_train.columns, label_names)].to_numpy()
Y_train = df_train[label_names].to_numpy()

## Define and Train a Network

In [56]:
## Start with standard cross-entropy loss (bpmll used later)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(14, activation = 'sigmoid')
])

optim_func = tf.keras.optimizers.Adagrad(
    learning_rate = 0.001, initial_accumulator_value = 0.1, epsilon = 1e-07,
    name = 'Adagrad')

model.compile(optimizer = optim_func,
              loss = 'categorical_crossentropy',
              )

In [57]:
model.fit(X_train, Y_train, epochs = 100)

Epoch 1/100
47/47 [==============================] - 0s 1ms/step - loss: 11.3453
Epoch 2/100
47/47 [==============================] - 0s 667us/step - loss: 11.2696
Epoch 3/100
47/47 [==============================] - 0s 636us/step - loss: 11.1830
Epoch 4/100
47/47 [==============================] - 0s 811us/step - loss: 11.0263
Epoch 5/100
47/47 [==============================] - 0s 675us/step - loss: 10.9696
Epoch 6/100
47/47 [==============================] - 0s 694us/step - loss: 10.9746
Epoch 7/100
47/47 [==============================] - 0s 684us/step - loss: 10.8280
Epoch 8/100
47/47 [==============================] - 0s 691us/step - loss: 10.8011
Epoch 9/100
47/47 [==============================] - 0s 647us/step - loss: 10.8454
Epoch 10/100
47/47 [==============================] - 0s 678us/step - loss: 10.9177
Epoch 11/100
47/47 [==============================] - 0s 659us/step - loss: 10.8435
Epoch 12/100
47/47 [==============================] - 0s 650us/step - loss: 10.9743
Epo